In [34]:
import pickle, os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import classification_report

In [35]:
data = os.path.abspath('../WESAD')

class subject:
    def __init__(self, path, num):
        self.name = f"S{num}"
        self.subject_keys = ['signal','label','subject']
        self.signal_keys = ['wrist']
        self.wrist_keys = ['BVP','EDA','ACC','TEMP']
        with open(os.path.join(path, self.name)+'/'+self.name+'.pkl','rb') as file:
            self.data = pickle.load(file, encoding='latin1')
        self.labels = self.data['label']
    
    def get_wrist_data(self):
        data = self.data['signal']['wrist']
        return data
    
    def get_subject_dataframe(self):
        wrist_data = self.get_wrist_data()
        bvp_signal = wrist_data['BVP'][:,0]
        eda_signal = wrist_data['EDA'][:,0]
        acc_x_signal = wrist_data['ACC'][:,0]
        acc_y_signal = wrist_data['ACC'][:,1]
        acc_z_signal = wrist_data['ACC'][:,2]
        temp_signal = wrist_data['TEMP'][:,0]
        # Upsampling data to match BVP data sampling rate using fourier method as described in Paper/dataset
        eda_upsampled = scipy.signal.resample(eda_signal, len(bvp_signal))
        temp_upsampled = scipy.signal.resample(temp_signal, len(bvp_signal))
        acc_x_upsampled = scipy.signal.resample(acc_x_signal, len(bvp_signal))
        acc_y_upsampled = scipy.signal.resample(acc_y_signal, len(bvp_signal))
        acc_z_upsampled = scipy.signal.resample(acc_z_signal, len(bvp_signal))
        label_df = pd.DataFrame(self.labels, columns=['label'])
        label_df.index = [(1 / 700) * i for i in range(len(label_df))] # 700 is the sampling rate of the label
        label_df.index = pd.to_datetime(label_df.index, unit='s')
        data_arrays = zip(bvp_signal, eda_upsampled, acc_x_upsampled, acc_y_upsampled, acc_z_upsampled, temp_upsampled)
        df = pd.DataFrame(data=data_arrays, columns=['BVP', 'EDA', 'ACC_x', 'ACC_y', 'ACC_z', 'TEMP'])
        df.index = [(1 / 64) * i for i in range(len(df))] # 64 = sampling rate of BVP
        df.index = pd.to_datetime(df.index, unit='s')
        df = df.join(label_df)
        df['label'] = df['label'].fillna(method='ffill')
        df.reset_index(drop=True, inplace=True)
        # df.drop(df[df['label'].isin([0.0, 4.0, 5.0, 6.0, 7.0])].index, inplace=True)
        # df['label'] = df['label'].replace([1.0, 2.0, 3.0], [0, 1, 0])
        df.reset_index(drop=True, inplace=True)
        return df 

s2 = subject(data,'2')
s2_wrist_data = s2.get_wrist_data()


In [36]:
subject_ids = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17] # ids for subjects in WESAD dataset
for subject_id in subject_ids:
    print(f'SUBJECT {subject_id}')
    df_subject = subject(data, subject_id).get_subject_dataframe()
    y = df_subject['label']
    df_subject.drop('label', axis=1, inplace=True)
    normalized_x=(df_subject-df_subject.min())/(df_subject.max()-df_subject.min())
    #x_train, x_test, y_train, y_test=train_test_split(df_subject,y,test_size=0.2)
    X_train, x_test, y_train, y_test=train_test_split(normalized_x,y,test_size=0.2)
    LDA= LinearDiscriminantAnalysis(solver = 'svd')
    y_out = LDA.fit(X_train, y_train).predict(x_test)
    #confusion_matrix(y_test, y_out)
    print(f'{classification_report(y_test, y_out, digits=4)}\n')

SUBJECT 2


/var/folders/lw/14dpn8b91z99svtdtfqnx09800tlp4/T/ipykernel_38055/717854677.py:39: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['label'] = df['label'].fillna(method='ffill')


              precision    recall  f1-score   support

         0.0     0.8680    0.7059    0.7786     38936
         1.0     0.7898    0.9083    0.8449     14818
         2.0     0.7469    0.9747    0.8457      7973
         3.0     0.6273    0.3644    0.4610      4523
         4.0     0.6258    0.9909    0.7671      9882
         6.0     0.2346    0.1180    0.1570       839
         7.0     0.0000    0.0000    0.0000       841

    accuracy                         0.7744     77812
   macro avg     0.5560    0.5803    0.5506     77812
weighted avg     0.7797    0.7744    0.7631     77812


SUBJECT 3


/var/folders/lw/14dpn8b91z99svtdtfqnx09800tlp4/T/ipykernel_38055/717854677.py:39: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['label'] = df['label'].fillna(method='ffill')
/Users/gnancy/.pyenv/versions/3.9.2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/gnancy/.pyenv/versions/3.9.2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/gnancy/.pyenv/versions/3.9.2/lib/python3.9/si

              precision    recall  f1-score   support

         0.0     0.7958    0.7050    0.7477     42689
         1.0     0.7112    0.9817    0.8248     14652
         2.0     0.6201    0.8430    0.7145      8176
         3.0     0.4689    0.2915    0.3595      4882
         4.0     0.8074    0.8773    0.8409     10045
         5.0     0.0000    0.0000    0.0000       895
         6.0     0.0000    0.0000    0.0000       891
         7.0     0.0000    0.0000    0.0000       881

    accuracy                         0.7413     83111
   macro avg     0.4254    0.4623    0.4359     83111
weighted avg     0.7203    0.7413    0.7225     83111


SUBJECT 4


...

/var/folders/lw/14dpn8b91z99svtdtfqnx09800tlp4/T/ipykernel_38055/717854677.py:39: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['label'] = df['label'].fillna(method='ffill')
/Users/gnancy/.pyenv/versions/3.9.2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/gnancy/.pyenv/versions/3.9.2/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/gnancy/.pyenv/versions/3.9.2/lib/python3.9/si

              precision    recall  f1-score   support

         0.0     0.8657    0.6786    0.7608     26674
         1.0     0.8381    0.9633    0.8963     15083
         2.0     0.7479    0.9146    0.8229      8808
         3.0     0.6489    0.9916    0.7845      4768
         4.0     0.8426    0.8476    0.8451     10114
         5.0     0.0000    0.0000    0.0000       644
         6.0     0.2888    0.3591    0.3201       596
         7.0     0.0000    0.0000    0.0000       539

    accuracy                         0.8063     67226
   macro avg     0.5290    0.5944    0.5537     67226
weighted avg     0.8049    0.8063    0.7964     67226


SUBJECT 16


/var/folders/lw/14dpn8b91z99svtdtfqnx09800tlp4/T/ipykernel_38055/717854677.py:39: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['label'] = df['label'].fillna(method='ffill')


              precision    recall  f1-score   support

         0.0     0.8234    0.7423    0.7808     31337
         1.0     0.8666    0.8146    0.8398     15211
         2.0     0.8745    0.9991    0.9326      8585
         3.0     0.6615    0.9782    0.7893      4769
         4.0     0.7885    0.9349    0.8555     10055
         5.0     0.0145    0.0014    0.0026       702
         6.0     0.0000    0.0000    0.0000       756
         7.0     0.0000    0.0000    0.0000       662

    accuracy                         0.8088     72077
   macro avg     0.5036    0.5588    0.5251     72077
weighted avg     0.7990    0.8088    0.7994     72077


SUBJECT 17
              precision    recall  f1-score   support

         0.0     0.6237    0.8115    0.7053     34676
         1.0     0.8938    0.9253    0.9092     15211
         2.0     0.6732    0.4792    0.5599      9408
         3.0     0.6937    0.9757    0.8108      4848
         4.0     0.0560    0.0016    0.0031      9405
         5.0

/var/folders/lw/14dpn8b91z99svtdtfqnx09800tlp4/T/ipykernel_38055/717854677.py:39: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['label'] = df['label'].fillna(method='ffill')


In [37]:
df2 = subject(data, 2).get_subject_dataframe()
df2[df2['label']== 0.0].count() #195904
df2[df2['label']== 1.0].count() #73216


/var/folders/lw/14dpn8b91z99svtdtfqnx09800tlp4/T/ipykernel_38055/717854677.py:39: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['label'] = df['label'].fillna(method='ffill')


BVP      73216
EDA      73216
ACC_x    73216
ACC_y    73216
ACC_z    73216
TEMP     73216
label    73216
dtype: int64